In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from pandas import Series, DataFrame
import numpy as np


In [2]:
def get_team_urls():
    team_urls=[]
    url = 'https://www.transfermarkt.us/premier-league/startseite/wettbewerb/GB1'
    headers = {"User-Agent":"Mozilla/5.0"}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    for link in soup.find_all("tr", class_='odd'):
        if link.a.get('href') != '#':
            team_urls.append('https://www.transfermarkt.us'+link.a.get('href')+'/plus/1')
    for link in soup.find_all("tr", class_='even'):
        if link.a.get('href') != '#':
            team_urls.append('https://www.transfermarkt.us'+link.a.get('href')+'/plus/1')
    return team_urls

In [3]:
def get_player_urls(team_url):
    player_urls=[]
    headers = {"User-Agent":"Mozilla/5.0"}
    page = requests.get(team_url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    for link in soup.find_all("td", class_='posrela'):
        player_urls.append('https://www.transfermarkt.us'+link.find("td", class_='hauptlink').a.get('href'))
    return player_urls

In [20]:
def get_player_profile(player_url):
    headers = {"User-Agent":"Mozilla/5.0"}
    page = requests.get(player_url, headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    profile = {}
    content = soup.find_all("span", class_="info-table__content")
    for i, element in enumerate(content):
        if i % 2 == 0:
            if content[i].text.replace('\n', '') == "Social-Media:":
                socials = content[i+1].find_all('a')
                socials_list = []
                for social in socials:
                    socials_list.append(social.get('href'))
                profile[content[i].text.replace('\n', '')] = socials_list
            else:
                profile[content[i].text.replace('\n', '')] = content[i+1].text.replace('\n', '')
    try:
        profile['Market Value'] = soup.find("div", class_="tm-player-market-value-development__current-value").text.replace('\n', '')
    except:
        profile['Market Value'] = np.nan
    full_name = player_url.split('https://www.transfermarkt.us/')[1].split('/profil')[0]
    if re.search('-', full_name):
            full_name = full_name.replace('-', ' ')
    full_name = full_name.title()
    profile['Full name:'] = full_name
    profile = DataFrame([profile])
    return profile

In [5]:
def get_player_stats(player_url):
    player_url = player_url.replace('profil', 'leistungsdatendetails')
    headers = {"User-Agent":"Mozilla/5.0"}
    page = requests.get(player_url, headers=headers)
    stats = pd.read_html(page.text)[1]
    return stats

In [6]:
team_urls = get_team_urls()
all_player_urls=[]
for team_url in team_urls:
    all_player_urls.append(get_player_urls(team_url))
all_player_urls

[['https://www.transfermarkt.us/ederson/profil/spieler/238223',
  'https://www.transfermarkt.us/stefan-ortega/profil/spieler/85941',
  'https://www.transfermarkt.us/scott-carson/profil/spieler/14555',
  'https://www.transfermarkt.us/ruben-dias/profil/spieler/258004',
  'https://www.transfermarkt.us/aymeric-laporte/profil/spieler/176553',
  'https://www.transfermarkt.us/john-stones/profil/spieler/186590',
  'https://www.transfermarkt.us/nathan-ake/profil/spieler/177476',
  'https://www.transfermarkt.us/sergio-gomez/profil/spieler/366930',
  'https://www.transfermarkt.us/benjamin-mendy/profil/spieler/157495',
  'https://www.transfermarkt.us/joao-cancelo/profil/spieler/182712',
  'https://www.transfermarkt.us/kyle-walker/profil/spieler/95424',
  'https://www.transfermarkt.us/rodri/profil/spieler/357565',
  'https://www.transfermarkt.us/kalvin-phillips/profil/spieler/351749',
  'https://www.transfermarkt.us/phil-foden/profil/spieler/406635',
  'https://www.transfermarkt.us/ilkay-gundogan/p

In [25]:
player_profile=DataFrame()
for team in all_player_urls:
    for player_url in team:
        player_profile = pd.concat([player_profile,get_player_profile(player_url)])
player_profile

,Full name:,Date of birth:,Place of birth:,Age:,Height:,Citizenship:,Position:,Foot:,Player agent:,Current club:,...,Date of last contract extension:,Outfitter:,Social-Media:,Market Value,Name in home country:,Contract option:,On loan from:,Contract there expires:,2nd club:,3nd club:
0,Ederson,"Aug 17, 1993",Osasco (SP),29,"1,88 m",Brazil Portugal,Goalkeeper ...,left,Gestifute,Manchester City,...,"Sep 1, 2021",Puma,"[http://twitter.com/edersonmoraes93, http://ww...",$49.50m,NaN,NaN,NaN,NaN,NaN,NaN
0,Stefan Ortega,"Nov 6, 1992",Hofgeismar,29,"1,85 m",Germany Spain,Goalkeeper ...,right,neblung sportsnetwork,Manchester City,...,NaN,NaN,[http://instagram.com/stefan.ortega1/],$6.60m,Stefan Ortega Moreno,NaN,NaN,NaN,NaN,NaN
0,Scott Carson,"Sep 3, 1985",Whitehaven,36,"1,88 m",England,Goalkeeper ...,right,Wasserman,Manchester City,...,"Jun 14, 2022",Puma,"[http://twitter.com/SCarsonOfficial, http://ww...",$330Th.,Scott Paul Carson,NaN,NaN,NaN,NaN,NaN
0,Ruben Dias,"May 14, 1997",Amadora,25,"1,87 m",Portugal,Defender - Centre-...,right,Gestifute,Manchester City,...,"Aug 30, 2021",Nike,"[http://twitter.com/rubendias, http://www.face...",$82.50m,Rúben Santos Gato Alves Dias,NaN,NaN,NaN,NaN,NaN
0,Aymeric Laporte,"May 27, 1994",Agen,28,"1,89 m",Spain France,Defender - Centre-...,left,Wasserman,Manchester City,...,"Feb 22, 2019",adidas,"[http://twitter.com/Laporte, http://www.facebo...",$49.50m,Aymeric Jean Louis Gerard Alphonse Laporte,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,Junior Stanislas,"Nov 26, 1989",London,32,"1,83 m",England St. Lucia,attack - Left Wing...,right,NaN,AFC Bournemouth,...,"Jul 21, 2021",Nike,[http://www.instagram.com/juniorstanislas/],$770Th.,Felix Junior Stanislas,NaN,NaN,NaN,NaN,NaN
0,David Brooks,"Jul 8, 1997",Warrington,25,"1,73 m",Wales England,attack - Right Win...,left,Unique Sports Group,AFC Bournemouth,...,"Mar 4, 2019",Nike,"[http://twitter.com/DRBrooks15, http://www.ins...",$17.60m,David Robert Brooks,NaN,NaN,NaN,NaN,NaN
0,Jamal Lowe,"Jul 21, 1994",London,28,"1,83 m",Jamaica England,attack - Right Win...,right,NaN,AFC Bournemouth,...,NaN,NaN,"[http://twitter.com/JamzLowe, http://www.insta...",$1.98m,Jamal Akua Lowe,Option for a further year,NaN,NaN,NaN,NaN
0,Dominic Solanke,"Sep 14, 1997",Basingstoke,24,"1,86 m",England Nigeria,attack - Centre-Fo...,right,Relatives,AFC Bournemouth,...,"Aug 3, 2022",adidas,"[http://twitter.com/DomSolanke, http://instagr...",$17.60m,Dominic Ayodele Solanke-Mitchell,NaN,NaN,NaN,NaN,NaN


In [67]:
get_player_stats('https://www.transfermarkt.us/mason-mount/profil/spieler/346483')

,Season,Competition,Appearances,Goals,Assists,Y/SY/R,Minutes Played
0,21/22,Premier League,32,11,10,4 / - / -,2.364'
1,21/22,FA Cup,5,1,3,- / - / -,474'
2,21/22,Champions League,7,1,2,- / - / -,538'
3,21/22,EFL Cup,6,-,1,- / - / -,304'
4,21/22,Club World Cup,2,-,-,- / - / -,49'
5,21/22,UEFA Super Cup,1,-,-,- / - / -,55'
6,20/21,Champions League,11,2,2,4 / - / -,761'
7,20/21,Premier League,36,6,6,2 / - / -,2.893'
8,20/21,FA Cup,5,1,-,- / - / -,408'
9,20/21,EFL Cup,2,-,1,- / - / -,180'


In [41]:
player_profile.to_csv('premierleague_profiles.csv', index=False, encoding='utf-8-sig')